In [2]:
from IPython.display import clear_output
!pip install XlsxWriter
clear_output()
print("[INFO] : Package Successfully Installed !!!")

[INFO] : Package Successfully Installed !!!


In [21]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Border, Side, Font, Alignment, PatternFill, numbers

# f = open('API_KEY.txt', 'r')


API_KEY = "b4021dc31a86d0c5bcd86955385f6714"

In [22]:
SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

BET_SIZE = 100

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
).json()

In [23]:
# odds_response
for i, oddsInfo in enumerate(odds_response):
  print(i,oddsInfo)


# print(f'{odds_response} \n')

0 {'id': '76ddd1c29b0cd79fc1d1f9864950a7a5', 'sport_key': 'baseball_mlb', 'sport_title': 'MLB', 'commence_time': '2023-06-18T17:05:00Z', 'home_team': 'Chicago Cubs', 'away_team': 'Baltimore Orioles', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-06-18T18:09:36Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-18T18:09:36Z', 'outcomes': [{'name': 'Baltimore Orioles', 'price': 1.69}, {'name': 'Chicago Cubs', 'price': 2.15}]}]}, {'key': 'pointsbetus', 'title': 'PointsBet (US)', 'last_update': '2023-06-18T18:10:04Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-18T18:10:04Z', 'outcomes': [{'name': 'Baltimore Orioles', 'price': 1.61}, {'name': 'Chicago Cubs', 'price': 2.25}]}]}, {'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-06-18T18:09:42Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-18T18:09:42Z', 'outcomes': [{'name': 'Baltimore Orioles', 'price': 1.6}, {'name': 'Chicago Cubs', 'price': 2.3}]}]}, {'key': 'mybookieag', 'tit

In [24]:
BOOKMAKER_INDEX = 0
NAME_INDEX = 1
ODDS_INDEX = 2
FIRST = 0

class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']

    def find_best_odds(self):
        # number of possible outcomes for a sporting event
        num_outcomes = len(self.data['bookmakers'][FIRST]['markets'][FIRST]['outcomes'])
        self.num_outcomes = num_outcomes

        # finding the best odds for each outcome in each event
        best_odds = [[None, None, float('-inf')] for _ in range(num_outcomes)]
        # [Bookmaker, Name, Price]

        bookmakers = event.data['bookmakers']
        for index, bookmaker in enumerate(bookmakers):

            # determing the odds offered by each bookmaker
            for outcome in range(num_outcomes):

                # determining if any of the bookmaker odds are better than the current best odds
                bookmaker_odds = float(bookmaker['markets'][FIRST]['outcomes'][outcome]['price'])
                current_best_odds = best_odds[outcome][ODDS_INDEX]

                if bookmaker_odds > current_best_odds:
                    best_odds[outcome][BOOKMAKER_INDEX] = bookmaker['title']
                    best_odds[outcome][NAME_INDEX] = bookmaker['markets'][FIRST]['outcomes'][outcome]['name']
                    best_odds[outcome][ODDS_INDEX] = bookmaker_odds

        self.best_odds = best_odds
        return best_odds

    def arbitrage(self):
        total_arbitrage_percentage = 0
        for odds in self.best_odds:
            total_arbitrage_percentage += (1.0 / odds[ODDS_INDEX])

        self.total_arbitrage_percentage = total_arbitrage_percentage
        self.expected_earnings = (BET_SIZE / total_arbitrage_percentage) - BET_SIZE

        # if the sum of the reciprocals of the odds is less than 1, there is opportunity for arbitrage
        if total_arbitrage_percentage < 1:
            return True
        return False

    # converts decimal/European best odds to American best odds
    def convert_decimal_to_american(self):
        best_odds = self.best_odds
        for odds in best_odds:
            decimal = odds[ODDS_INDEX]
            if decimal >= 2:
                american = (decimal - 1) * 100
            elif decimal < 2:
                american = -100 / (decimal - 1)
            odds[ODDS_INDEX] = round(american, 2)
        return best_odds

    def calculate_arbitrage_bets(self):
        bet_amounts = []
        for outcome in range(self.num_outcomes):
            individual_arbitrage_percentage = 1 / self.best_odds[outcome][ODDS_INDEX]
            bet_amount = (BET_SIZE * individual_arbitrage_percentage) / self.total_arbitrage_percentage
            bet_amounts.append(round(bet_amount, 2))

        self.bet_amounts = bet_amounts
        return bet_amounts

In [25]:
events = []
for data in odds_response:
    events.append(Event(data))
    # print(data)
    # print()

arbitrage_events = []
for event in events:
    best_odds = event.find_best_odds()
    if event.arbitrage():
        arbitrage_events.append(event)

for event in arbitrage_events:
    event.calculate_arbitrage_bets()
    event.convert_decimal_to_american()

In [26]:
MAX_OUTCOMES = max([event.num_outcomes for event in arbitrage_events])
ARBITRAGE_EVENTS_COUNT = len(arbitrage_events)

my_columns = ['ID', 'Sport Key', 'Expected Earnings'] + list(np.array([[f'Bookmaker #{outcome}', f'Name #{outcome}', f'Odds #{outcome}', f'Amount to Buy #{outcome}'] for outcome in range(1, MAX_OUTCOMES + 1)]).flatten())
dataframe = pd.DataFrame(columns=my_columns)

In [27]:
for event in arbitrage_events:
    # print(event.best_odds)
    row = []
    row.append(event.id)
    row.append(event.sport_key)
    row.append(round(event.expected_earnings, 2))
    for index, outcome in enumerate(event.best_odds):
        row.append(outcome[BOOKMAKER_INDEX])
        row.append(outcome[NAME_INDEX])
        row.append(outcome[ODDS_INDEX])
        row.append(event.bet_amounts[index])
    while len(row) < len(dataframe.columns):
        row.append('N/A')
    dataframe.loc[len(dataframe.index)] = row

In [28]:
dataframe

,ID,Sport Key,Expected Earnings,Bookmaker #1,Name #1,Odds #1,Amount to Buy #1,Bookmaker #2,Name #2,Odds #2,Amount to Buy #2
0,76ddd1c29b0cd79fc1d1f9864950a7a5,baseball_mlb,1.64,DraftKings,Baltimore Orioles,-144.93,60.14,MyBookie.ag,Chicago Cubs,155.00,39.86
1,c997596bf5e345dbf6d2721cb4dea022,baseball_mlb,15.01,Unibet,New York Mets,225.00,35.39,Circa Sports,St. Louis Cardinals,-128.21,64.61
2,58429b0f57cc5e78077352f7758f90da,baseball_ncaa,1.43,BetUS,TCU Horned Frogs,190.00,34.98,DraftKings,Virginia Cavaliers,-178.57,65.02
3,6a359b5da8ce70675abff78bbacb6a0e,baseball_mlb,0.30,WynnBET,Milwaukee Brewers,-163.93,62.30,LowVig.ag,Pittsburgh Pirates,166.00,37.70
4,9977df68f83bbab06882f5b444aa5f09,baseball_mlb,0.16,BetRivers,Texas Rangers,-135.14,57.56,LowVig.ag,Toronto Blue Jays,136.00,42.44
5,886ce99dcfddeed7025a5f612890bed0,basketball_wnba,0.23,DraftKings,Chicago Sky,215.00,31.82,LowVig.ag,Washington Mystics,-212.77,68.18


In [29]:
writer = pd.ExcelWriter('bets.xlsx')
dataframe.to_excel(writer, index=False)
writer.save()

<ipython-input-29-b930a8caff5b>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [30]:
BLACK = '000000'
LIGHT_GREY = 'D6D6D6'
DARK_GREY = '9F9F9F'
RED = 'FEA0A0'
BLUE = 'A0CEFE'
YELLOW = 'FFE540'

COLORS = [RED, BLUE]

ID_COLUMN_FILL = PatternFill(fill_type='solid', start_color=DARK_GREY, end_color=DARK_GREY)
SPORT_KEY_COLUMN_FILL = PatternFill(fill_type='solid', start_color=LIGHT_GREY, end_color=LIGHT_GREY)
EXPECTED_EARNINGS_COLUMN_FILL = PatternFill(fill_type='solid', start_color=YELLOW, end_color=YELLOW)

CENTER_ALIGNMENT = Alignment(horizontal='center', vertical='bottom', indent=0)

TOP_ROW_BORDER = Border(bottom=Side(border_style='thick', color=BLACK))
NORMAL_ROW_BORDER = Border(top=Side(border_style='thin', color=LIGHT_GREY), bottom=Side(border_style='thin', color=DARK_GREY))

wb = load_workbook('bets.xlsx')
ws = wb.active
ws.title = 'Upcoming'
# changing width
for col in range(1, 26):
    ws.column_dimensions[chr(col + 64)].width = 20

for cell in ws['A']:
    cell.fill = ID_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT

for cell in ws['B']:
    cell.fill = SPORT_KEY_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT

for cell in ws['C']:
    cell.fill = EXPECTED_EARNINGS_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    cell.number_format = numbers.BUILTIN_FORMATS[7]

START_INDEX = 'D'
for index in range(MAX_OUTCOMES):
    for col in ws[START_INDEX : chr(ord(START_INDEX) + 3)]:
        for cell in col:
            color = COLORS[int(index % 2)]
            cell.fill = PatternFill(fill_type='solid', start_color=color, end_color=color)
            cell.alignment = CENTER_ALIGNMENT
            if cell.column % 4 == 3:
                cell.number_format = numbers.BUILTIN_FORMATS[7]

    START_INDEX = chr(ord(START_INDEX) + 4)

for cell in ws['1']:
    cell.border = TOP_ROW_BORDER

for row in range(2, ARBITRAGE_EVENTS_COUNT + 2):
    for cell in ws[str(row)]:
        cell.border = NORMAL_ROW_BORDER

wb.save('upcoming_events_bets.xlsx')

In [32]:
final_betting = pd.read_excel("upcoming_events_bets.xlsx")
final_betting

,ID,Sport Key,Expected Earnings,Bookmaker #1,Name #1,Odds #1,Amount to Buy #1,Bookmaker #2,Name #2,Odds #2,Amount to Buy #2
0,76ddd1c29b0cd79fc1d1f9864950a7a5,baseball_mlb,1.64,DraftKings,Baltimore Orioles,-144.93,60.14,MyBookie.ag,Chicago Cubs,155.00,39.86
1,c997596bf5e345dbf6d2721cb4dea022,baseball_mlb,15.01,Unibet,New York Mets,225.00,35.39,Circa Sports,St. Louis Cardinals,-128.21,64.61
2,58429b0f57cc5e78077352f7758f90da,baseball_ncaa,1.43,BetUS,TCU Horned Frogs,190.00,34.98,DraftKings,Virginia Cavaliers,-178.57,65.02
3,6a359b5da8ce70675abff78bbacb6a0e,baseball_mlb,0.30,WynnBET,Milwaukee Brewers,-163.93,62.30,LowVig.ag,Pittsburgh Pirates,166.00,37.70
4,9977df68f83bbab06882f5b444aa5f09,baseball_mlb,0.16,BetRivers,Texas Rangers,-135.14,57.56,LowVig.ag,Toronto Blue Jays,136.00,42.44
5,886ce99dcfddeed7025a5f612890bed0,basketball_wnba,0.23,DraftKings,Chicago Sky,215.00,31.82,LowVig.ag,Washington Mystics,-212.77,68.18
